[Kaggle - Titanic - Tabular data - Binary Classification](https://www.kaggle.com/c/titanic/data)<br>
[Read csv from notebook 20200511_EDA_DataPreprocess](./20200511_EDA_DataPreprocess.ipynb)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

SEED = 1999

In [2]:
df_train_combined = pd.read_csv('./input/df_train_combined.csv')
df_test_combined = pd.read_csv('./input/df_test_combined.csv')

display(df_train_combined.head())
print(df_train_combined.shape)
display(df_test_combined.head())
print(df_test_combined.shape)

,Survived,Age,SibSp,Parch,Fare,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Title_Master.,Title_Miss.,Title_Mr.,Title_Mrs.,Title_other
0,0,-0.592481,0.432793,-0.473674,-0.502445,0,0,1,0,1,0,0,1,0,0,1,0,0
1,1,0.638789,0.432793,-0.473674,0.786845,1,0,0,1,0,1,0,0,0,0,0,1,0
2,1,-0.284663,-0.474545,-0.473674,-0.488854,0,0,1,1,0,0,0,1,0,1,0,0,0
3,1,0.407926,0.432793,-0.473674,0.420730,1,0,0,1,0,0,0,1,0,0,0,1,0
4,0,0.407926,-0.474545,-0.473674,-0.486337,0,0,1,0,1,0,0,1,0,0,1,0,0


(891, 18)


,Age,SibSp,Parch,Fare,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Title_Master.,Title_Miss.,Title_Mr.,Title_Mrs.,Title_other
0,0.369449,-0.474545,-0.473674,-0.490783,0,0,1,0,1,0,1,0,0,0,1,0,0
1,1.331378,0.432793,-0.473674,-0.507479,0,0,1,1,0,0,0,1,0,0,0,1,0
2,2.485693,-0.474545,-0.473674,-0.453367,0,1,0,0,1,0,1,0,0,0,1,0,0
3,-0.207709,-0.474545,-0.473674,-0.474005,0,0,1,0,1,0,0,1,0,0,1,0,0
4,-0.592481,0.432793,0.767630,-0.401017,0,0,1,1,0,0,0,1,0,0,0,1,0


(418, 17)


---

# First of all, train_test_split

In [3]:
from sklearn.model_selection import train_test_split

X = df_train_combined.drop('Survived', axis=1)
y = df_train_combined['Survived']

X_train, X_dev, y_train, y_dev = train_test_split(X, y,
                                                  test_size = 0.2,
                                                  stratify = y,
                                                  random_state = SEED)

print('X_train.shape:', X_train.shape)
print('y_train.shape:', y_train.shape)
print('X_dev.shape:', X_dev.shape)
print('y_dev.shape:', y_dev.shape)

X_train.shape: (712, 17)
y_train.shape: (712,)
X_dev.shape: (179, 17)
y_dev.shape: (179,)


---

# 1) Test for different models using CV (find the best algorithm)

In [4]:
# Models
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

# Cross Validation
from sklearn.model_selection import cross_validate

In [5]:
models = [LogisticRegression(),
          GaussianNB(),
          KNeighborsClassifier(),
          SVC(),
          RandomForestClassifier(),
          XGBClassifier()]

# Perform cv across models
cv_result={}
for model in models:
    cv_result[model.__class__.__name__] = cross_validate(model, X, y,
                                                         cv=3,
                                                         scoring='accuracy',
                                                         return_train_score=1,
                                                         verbose=1)

display(cv_result)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s finished


{'LogisticRegression': {'fit_time': array([0.01316833, 0.01101255, 0.01097703]),
  'score_time': array([0.00099921, 0.0009985 , 0.00099921]),
  'test_score': array([0.81481481, 0.81818182, 0.82491582]),
  'train_score': array([0.84343434, 0.82491582, 0.83164983])},
 'GaussianNB': {'fit_time': array([0.0019989 , 0.0019989 , 0.00199866]),
  'score_time': array([0.00099945, 0.00099945, 0.00099945]),
  'test_score': array([0.76767677, 0.81481481, 0.81481481]),
  'train_score': array([0.8030303 , 0.80808081, 0.80808081])},
 'KNeighborsClassifier': {'fit_time': array([0.00299835, 0.00299811, 0.00299859]),
  'score_time': array([0.00999427, 0.01499033, 0.0109942 ]),
  'test_score': array([0.78787879, 0.80808081, 0.82828283]),
  'train_score': array([0.87542088, 0.85353535, 0.85185185])},
 'SVC': {'fit_time': array([0.00999451, 0.00899601, 0.0109961 ]),
  'score_time': array([0.00299811, 0.00299811, 0.00299811]),
  'test_score': array([0.81818182, 0.84175084, 0.84511785]),
  'train_score': arr

In [6]:
cv_result_df = pd.DataFrame(cv_result).T
cv_result_df = cv_result_df.applymap(lambda x: np.mean(x))   # apply np.mean to each cell

display(cv_result_df)

,fit_time,score_time,test_score,train_score
LogisticRegression,0.011719,0.000999,0.819304,0.833333
GaussianNB,0.001999,0.000999,0.799102,0.806397
KNeighborsClassifier,0.002998,0.011993,0.808081,0.860269
SVC,0.009996,0.002998,0.835017,0.842873
RandomForestClassifier,0.140592,0.011328,0.797980,0.988215
XGBClassifier,0.047342,0.001332,0.819304,0.887767


**SVC has the best performance**

---

# 2) GridSearch over SVC (find the best hyperparams)

In [10]:
from sklearn.model_selection import GridSearchCV

param_grid = [
    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]},
    {'kernel': ['rbf'], 'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001]},
]

svc = SVC()
clf = GridSearchCV(svc,
                   param_grid,
                   scoring='accuracy',
                   cv=3,
                   return_train_score=True)
clf.fit(X, y)

# save best model
best_model = []
best_model.append(clf.best_estimator_)

print('best_params:', clf.best_params_)
print('best_score:', clf.best_score_)
print('best_estimator:', clf.best_estimator_)

print('\nmean_train_score:', clf.cv_results_['mean_train_score'][clf.best_index_])
print('std_train_score:', clf.cv_results_['std_train_score'][clf.best_index_])
print('mean_test_score:', clf.cv_results_['mean_test_score'][clf.best_index_])
print('std_test_score:', clf.cv_results_['std_test_score'][clf.best_index_])

best_params: {'C': 1000, 'gamma': 0.001, 'kernel': 'rbf'}
best_score: 0.8282828282828284
best_estimator: SVC(C=1000, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

mean_train_score: 0.8350168350168351
std_train_score: 0.005991626404057958
mean_test_score: 0.8282828282828284
std_test_score: 0.009523323652344083


In [11]:
from sklearn.model_selection import GridSearchCV

param_grid = [
    {'kernel': ['rbf'], 'C': [1000, 2000, 3000], 'gamma': [0.01, 0.001, 0.0005]},
]

svc = SVC()
clf = GridSearchCV(svc,
                   param_grid,
                   scoring='accuracy',
                   cv=3,
                   return_train_score=True)
clf.fit(X, y)

# save best model
best_model.append(clf.best_estimator_)

print('best_params:', clf.best_params_)
print('best_score:', clf.best_score_)
print('best_estimator:', clf.best_estimator_)

print('\nmean_train_score:', clf.cv_results_['mean_train_score'][clf.best_index_])
print('std_train_score:', clf.cv_results_['std_train_score'][clf.best_index_])
print('mean_test_score:', clf.cv_results_['mean_test_score'][clf.best_index_])
print('std_test_score:', clf.cv_results_['std_test_score'][clf.best_index_])

best_params: {'C': 2000, 'gamma': 0.001, 'kernel': 'rbf'}
best_score: 0.8305274971941637
best_estimator: SVC(C=2000, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

mean_train_score: 0.8355780022446689
std_train_score: 0.007053762676754521
mean_test_score: 0.8305274971941637
std_test_score: 0.010408101566212915


In [12]:
from sklearn.model_selection import GridSearchCV

param_grid = [
    {'kernel': ['rbf'], 'C': [1500, 2000, 2500], 'gamma': [0.005 ,0.001]},
]

svc = SVC()
clf = GridSearchCV(svc,
                   param_grid,
                   scoring='accuracy',
                   cv=3,
                   return_train_score=True)
clf.fit(X, y)

# save best model
best_model.append(clf.best_estimator_)

print('best_params:', clf.best_params_)
print('best_score:', clf.best_score_)
print('best_estimator:', clf.best_estimator_)

print('\nmean_train_score:', clf.cv_results_['mean_train_score'][clf.best_index_])
print('std_train_score:', clf.cv_results_['std_train_score'][clf.best_index_])
print('mean_test_score:', clf.cv_results_['mean_test_score'][clf.best_index_])
print('std_test_score:', clf.cv_results_['std_test_score'][clf.best_index_])

best_params: {'C': 2500, 'gamma': 0.005, 'kernel': 'rbf'}
best_score: 0.830527497194164
best_estimator: SVC(C=2500, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.005, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

mean_train_score: 0.8540965207631874
std_train_score: 0.008398782012960548
mean_test_score: 0.830527497194164
std_test_score: 0.012991960609192164


**Best model is the second attempt (best_model[1]), best params: {'C': 2000, 'gamma': 0.001, 'kernel': 'rbf'}<br>
Because less overfitting**

---

# 3) Prediction on testing set & Submission

In [21]:
# Prediction

clf = best_model[1]

y_pred = clf.predict(df_test_combined)

print(y_pred)

[0 1 0 0 1 0 1 0 1 0 0 0 1 0 1 1 0 0 1 1 0 1 1 0 1 0 1 0 0 0 0 0 1 1 0 0 1
 1 0 0 0 0 0 1 1 0 0 0 1 1 0 0 1 1 0 0 0 0 0 1 0 0 0 1 1 1 1 0 0 1 1 0 1 0
 1 0 0 1 0 1 1 0 0 0 0 0 1 1 1 1 1 0 1 0 0 0 1 0 1 0 1 0 0 0 1 0 0 0 0 0 0
 1 1 1 1 0 0 1 0 1 1 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0
 0 0 1 0 0 1 0 0 1 1 0 1 1 1 1 0 0 1 0 0 1 1 0 0 0 0 0 1 1 0 1 1 0 0 1 0 1
 0 1 0 0 0 0 0 1 0 1 0 1 1 0 1 1 1 0 1 0 0 1 0 1 0 0 0 0 1 0 0 1 0 1 0 1 0
 1 0 1 1 0 1 0 0 0 1 0 0 0 0 0 0 1 1 1 1 0 0 1 0 1 0 1 1 1 0 0 0 0 0 0 0 1
 0 0 0 1 1 0 0 0 0 1 0 0 0 1 1 0 1 0 0 0 0 1 1 1 1 1 0 0 0 0 0 0 1 0 0 0 0
 1 0 0 0 0 0 0 0 1 1 0 1 0 1 0 0 0 1 1 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0 0
 1 0 0 0 0 0 1 0 0 0 1 1 1 0 1 0 1 1 0 0 0 1 0 1 0 0 1 0 1 1 0 1 0 0 1 1 0
 0 1 0 0 1 1 1 0 0 0 0 0 1 1 0 1 0 0 0 0 0 1 1 0 0 1 0 1 0 0 1 0 1 0 0 0 0
 0 1 1 1 1 1 0 1 0 0 1]


In [23]:
# Submission

submission = pd.read_csv("./input/raw/gender_submission.csv", index_col='PassengerId')
submission['Survived'] = y_pred
submission.to_csv('./output/submission_titanic_20200513.csv')